### Stimulus encoding using MI estimator

In [ ]:
import sys; sys.path.insert(1, '/home/vinicius/storage1/projects/GrayData-Analysis')
import os

from   GDa.session           import session_info, session
from   GDa.util              import create_stages_time_grid
from   GDa.net.util          import compute_coherence_thresholds
from   GDa.temporal_network  import temporal_network
from   GDa.stats.bursting    import *

from   xfrites.conn.conn_coh import conn_coherence_wav
from   xfrites.conn.conn_tf  import wavelet_spec

from   mne.viz               import circular_layout, \
                                    plot_connectivity_circle

import matplotlib.pyplot     as     plt
import matplotlib
import GDa.graphics.plot     as     plot

import numpy             as np
import xarray            as xr
import matplotlib.pyplot as plt
import h5py
import scipy
from   tqdm              import tqdm

In [ ]:
SMALL_SIZE, MEDIUM_SIZE, BIGGER_SIZE = plot.set_plot_config()

In [ ]:
# Create directory to save figures
if not os.path.exists("img/n6.0.3"):
    os.makedirs("img/n6.0.3")

In [ ]:
# Extract area names
def _extract_roi(roi, sep):
    # Code by Etiene
    x_s, x_t = [], []
    for r in roi:
        _x_s, _x_t = r.split(sep)
        x_s.append(_x_s), x_t.append(_x_t)
    return np.array(x_s), np.array(x_t)

#### Load session and filter for ODRT trials with correct response

In [ ]:
stages      = ['baseline', 'cue', 'delay', 'match']
band_names  = [r'$\theta$', r'$\alpha$', r'$\beta$', r'high-$\beta$ ', r'$\gamma$' ]

In [ ]:
# Instantiate class
ses = session(raw_path = '../GrayLab/', monkey = 'lucy', date = '150128', session = 1,
              slvr_msmod = False, align_to = 'cue', evt_dt = [-0.65, 3.00])

# Read data from .mat files
ses.read_from_mat()

In [ ]:
# Filtering by trials
data = ses.filter_trials(trial_type=[1],behavioral_response=[1])

#### Compute spectra for the data

In [ ]:
# Smoothing windows
sm_times  = 0.5
sm_freqs  = 1
sm_kernel = "square"

# Defining parameters
delta = 15       # Downsampling factor
mode  = 'morlet' # ("morlet", "mt_1", "mt_2")
#  mode  = 'multitaper' # ("morlet", "mt_1", "mt_2")
foi   = np.array([
        [0.1, 6.],
        [6., 14.],
        [14., 26.],
        [26., 42.],
        [42., 80.]
            ])

n_freqs  = 50
freqs    = np.linspace(foi[0,0], foi[-1,1], n_freqs, endpoint=True)
n_cycles = freqs/2

In [ ]:
sxx = wavelet_spec(data, freqs=freqs, roi=data.roi, times="time", sfreq=data.attrs["fsample"],
                 foi=foi, sm_times=sm_times, sm_freqs=sm_freqs, sm_kernel=sm_kernel, mode=mode, 
                 n_cycles=n_cycles, mt_bandwidth=None, decim=delta, kw_cwt={}, 
                 kw_mt={}, block_size=1, n_jobs=20, verbose=None)

#### Get stage mask for this session

In [ ]:
s_mask = create_stages_time_grid(sxx.attrs["t_cue_on"], sxx.attrs["t_cue_off"],
                        sxx.attrs["t_match_on"], sxx.attrs["fsample"],
                        sxx.times.values, sxx.sizes["trials"], flatten=False)

#### Get average power per task stage

In [ ]:
tmp = []
for i, s in enumerate(stages):
    tmp += [(sxx*s_mask[s][:,None,None,:]).sum(dim="times")/s_mask[s][:,None,None,:].sum(-1)]

In [ ]:
tmp = xr.concat( tmp, dim="times" )
tmp.attrs = sxx.attrs
tmp = tmp.assign_coords({"times":[0,1,2,3]})

In [ ]:
tmp = tmp.transpose("trials","roi","freqs","times")

In [ ]:
sxx

### MI workflow

In [ ]:
from frites.dataset  import SubjectEphy, DatasetEphy
from frites.workflow import WfMi

In [ ]:
dt = DatasetEphy([sxx], y=[sxx.attrs["stim"].astype(int)], times="times", roi=None, agg_ch=False)

In [ ]:
mi_type   = 'cd'
inference = 'ffx'
kernel    = np.hanning(1)
wf        = WfMi(mi_type, inference, verbose=True, kernel=kernel)

In [ ]:
# list of corrections for multiple comparison
mcps = ['cluster', 'maxstat', 'fdr', 'bonferroni']
kw = dict(n_jobs=20, n_perm=200)
"""
The `cluster_th` input parameter specifies how the threshold is defined.
Use either :
* a float for a manual threshold
* None and it will be infered using the distribution of permutations
* 'tfce' for a TFCE threshold
"""
cluster_th = 'tfce'  # {float, None, 'tfce'}

mi, pvalues = wf.fit(dt, mcp="cluster", cluster_th=cluster_th, **kw)
    # remove p-values that exceed 0.05
    #pvalues.data[pvalues >= .05] = 1.

    #plt.subplot(212)
    #plt.plot(pvalues.squeeze(), label=f'ffx-{mcp}')
    #plt.xlabel('Time (s)'), plt.ylabel("P-values")
    #plt.title("P-values (ffx)")
    #plt.autoscale(tight=True)
#plt.legend()

#plt.subplot(211)
#plt.plot( mi.squeeze())
#plt.xlabel('Time (s)'), plt.ylabel("Bits")
#plt.title("Mutual information I(C; D)")
#plt.autoscale(tight=True)

plt.show()

In [ ]:
plt.figure(figsize=(20,4))
for i in range(5):
    plt.subplot(1,5,i+1)
    extent=[pvalues.times.values[0], mi.times.values[-1], 0,22]
    plt.imshow(mi.isel(freqs=i).T, aspect="auto", cmap="jet", origin="lower", vmax=0.04, vmio)
    plt.colorbar()

In [ ]:
mi_c = mi.copy()
mi_c.data[pvalues.data >= 0.05] = np.nan

mi_c.plot(x='times', y='freqs', col='roi')

In [ ]:
plt.plot(mi.times, np.nanmean(mi*(pvalues<=0.05),-1).T )
plt.legend(band_names)
plt.ylabel("MI (bits)")
plt.xlabel("time")

In [ ]:
#### Original data
net = temporal_network(coh_file=f'super_tensor_k_{0.5}_surr_False_morlet.nc', monkey='lucy', 
                       session=1, date='150128', trial_type=[1],
                       behavioral_response=[1], wt=(20,20), drop_trials_after=True,
                       relative=True, verbose=True, q=None, keep_weights=False)

In [ ]:
out = net.super_tensor.transpose("trials","roi","freqs","times")

In [ ]:
dt = DatasetEphy([out], y=[sxx.attrs["stim"].astype(int)], times="times", roi=None, agg_ch=False)

In [ ]:
mi_type   = 'cd'
inference = 'ffx'
kernel    = np.hanning(1)
wf        = WfMi(mi_type, inference, verbose=True, kernel=kernel)

In [ ]:
# list of corrections for multiple comparison
mcps = ['cluster', 'maxstat', 'fdr', 'bonferroni']
kw = dict(n_jobs=20, n_perm=200)
"""
The `cluster_th` input parameter specifies how the threshold is defined.
Use either :
* a float for a manual threshold
* None and it will be infered using the distribution of permutations
* 'tfce' for a TFCE threshold
"""
cluster_th = 'tfce'  # {float, None, 'tfce'}
mi, pvalues = wf.fit(dt, mcp="cluster", cluster_th=cluster_th, **kw)

In [ ]:
plt.figure(figsize=(20,4))
for i in range(5):
    plt.subplot(1,5,i+1)
    extent=[pvalues.times.values[0], mi.times.values[-1], 0,22]
    plt.imshow(mi.isel(freqs=i).T, aspect="auto", cmap="jet", origin="lower", vmax=0.04, vmin=0)
    plt.colorbar()

In [ ]:
mi.sizes

In [ ]:
plt.plot(mi.times, np.nanmean(mi*(pvalues<=0.05),-1).T )
plt.legend(band_names)
plt.ylabel("MI (bits)")
plt.xlabel("time")

In [ ]:
.shape